In [1]:
def process_raster_statistics(stats, gdf, raster_name, project_name, flight, more_stat_percent):
    """
    Processes raster statistics by extracting raw values, computing additional statistics,
    and formatting the final DataFrame with necessary metadata.

    Parameters:
        stats (list of dict): List of dictionaries containing raster statistics.
        gdf (GeoDataFrame): Geospatial dataframe containing feature IDs.
        raster_name (str): Name of the raster being processed.
        project_name (str): Project identifier.
        flight (str): Flight identifier.
        more_stat_percent (list): List of percent values for additional statistics.

    Returns:
        DataFrame: Processed statistics DataFrame.
    """
    # Extract raw values for additional calculations
    raw_values = [
        stat["mini_raster_array"].compressed() if "mini_raster_array" in stat else np.array([])
        for stat in stats
    ]
    
    additional_stats = [compute_additional_statistics(vals, more_stat_percent) for vals in raw_values]
    
    # Create DataFrame from stats and drop unwanted columns
    df_stats = pd.DataFrame(stats).drop(columns=["mini_raster_array", "mini_raster_affine", "mini_raster_nodata"], errors="ignore")
    df_stats.columns = STAT_ORDER[:10]  # Assign expected column order

    # Convert additional statistics into DataFrame
    df_additional = pd.DataFrame(additional_stats)

    # Merge basic and additional statistics
    df = pd.concat([df_stats, df_additional], axis=1)
    
    # Prefix raster name to stat columns
    df.columns = [f"{raster_name}_{col}" for col in df.columns]
    
    # Insert metadata columns
    df.insert(0, "id", gdf.id)
    df.insert(1, "project", project_name)
    df.insert(2, "flight", flight)

    return df

In [ ]:
processed_df = process_raster_statistics(stats, gdf, "ndvi", "MyProject", "Flight_123", MORE_STAT_PERCENT)
all_results.append(processed_df)
